0) Carga de datos, librerias y funciones

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt', download_dir='nltk_data')
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
import re
import html
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from transformers import pipeline

[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
ruta='data/resultados_enquestes.csv'
df=pd.read_csv(ruta,sep=',')
df.head(5)


,ID,ENQUESTA_ID,SEMESTRE,PREGUNTA_ID,RESPOSTA_ID,RESPOSTA,IDP,SUMATORI,TIPUS_PREGUNTA,ENQUESTA_CREATED_AT,PREGUNTA_CREATED_AT,RESPOSTA_CREATED_AT,CREATION_DATE,UPDATE_DATE
0,8774167,668,20232.0,17675,426985,SolicitÃ© un cambio de idioma de docencia y de...,1680015,1,PREGUNTA_OBERTA,2024-03-01 09:14:33,2024-03-01 09:14:40,2024-03-27 16:04:38,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
1,8774168,670,NaN,17738,427070,MejorarÃ­a la velocidad de respuesta por parte...,1265565,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 17:38:19,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
2,8774170,670,NaN,17738,427313,Algunas asignaturas estÃ¡n en inglÃ©s porque s...,1418238,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 19:49:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
3,8774171,670,NaN,17738,427349,A Canvas resulta poc intuÃ¯tiu que els recurso...,1414112,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:10:34,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000
4,8774172,670,NaN,17738,427421,"Se necesitan mas profesores vocacionales, y un...",815659,1,PREGUNTA_OBERTA,2024-05-22 09:02:23,2024-05-22 09:02:32,2024-06-12 20:52:45,2024-08-09 09:49:23.133000,2024-08-09 09:49:23.133000


1) Preprocesado

In [ ]:
text_column_name ='RESPOSTA'
text_data = df[text_column_name].tolist()
print(type(text_data))
text_data

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ''  
    text = html.unescape(text)
    text = text.lower()
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    
    return text

cleaned_data = [clean_text(doc) for doc in text_data]
cleaned_data

In [95]:
stopwords_catalan = [
    'i', 'de', 'a', 'la', 'el', 'en', 'que', 'no', 'un', 'per', 'amb', 'els', 'les', 'dels', 'al', 'del', 'com', 'una', 'o', 'a', 'abans', 'ací', 'això', 'així', 'al', 'alguna', 'algunes', 'alguns', 'allà', 'allí', 
    'allò', 'als', 'altra', 'altre', 'altres', 'amb', 'aquell', 'aquella', 'aquelles', 'aquells', 
    'aquest', 'aquesta', 'aquestes', 'aquests', 'ben', 'cada', 'com', 'contra'
]

In [96]:
def dtm_(cleaned_data):
    vectorizer = CountVectorizer(stop_words=stopwords_catalan, token_pattern=r'\b[a-zA-Z]+\b', min_df=10)
    dtm = vectorizer.fit_transform(cleaned_data)
    return dtm, vectorizer

dtm, vectorizer=dtm_(cleaned_data)

2) Tópicos de Palabras

In [ ]:
def lda_find_n_v1(dtm):
    search_params = {'n_components': [4, 5, 6, 7, 8, 9, 10,11,12,13,14,15]}
    lda = LatentDirichletAllocation()
    model = GridSearchCV(lda, param_grid=search_params)
    model.fit(dtm)
    print("Mejor número de temas:", model.best_params_)
    print("Log-Likelihood del mejor modelo:", model.best_score_)

lda_find_n_v1(dtm)

In [ ]:
def lda_find_n_v2(dtm):
    dtm_train, dtm_test = train_test_split(dtm, test_size=0.25, random_state=42)
    lda = LDA(n_components=7, random_state=42)
    lda.fit(dtm_train)
    perplexity_train = lda.perplexity(dtm_test)
    print(perplexity_train)
    k_values = [2, 4, 8, 16]
    perplexities = []

    for k in k_values:
        print(k)
        lda = LDA(n_components=k, random_state=42)
        lda.fit(dtm_train)
        perplexity = lda.perplexity(dtm_test)
        perplexities.append(perplexity)

    plt.plot(k_values, perplexities, marker='o')
    plt.xlabel('Número de temas (k)')
    plt.ylabel('Perplejidad')
    plt.title('Perplejidad para diferentes valores de k')
    plt.show()

lda_find_n_v2(dtm)

In [ ]:
def lda(n, vectorizer):
    lda = LDA(n_components=n, random_state=42)
    lda.fit(dtm)
    terms = vectorizer.get_feature_names_out()
    topic_words = {}
    for topic_idx, topic in enumerate(lda.components_):
        top_terms_idx = topic.argsort()[-40:][::-1]  # Obtener los índices de las 40 palabras más importantes
        top_terms = [terms[i] for i in top_terms_idx]
        topic_words[topic_idx] = top_terms

    print(topic_words)

    return topic_words

lda(4, vectorizer)

3) Analisis de Sentimiento

In [6]:
df=df.sample(5)
comentarios=df['RESPOSTA'].tolist()

In [ ]:
from transformers import logging
logging.set_verbosity(logging.WARNING)
import torch
print(torch.__version__)
import tensorflow as tf
print(tf.__version__)



In [ ]:
classifier=pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
sentimiento_predicho=classifier(comentarios)
for comentarios, sentimiento_predicho in zip(comentarios, sentimiento_predicho):
  print(f"comentario: {comentarios}\nPredicted Sentiment: {sentimiento_predicho['label']} (Confidence: {sentimiento_predicho['score']:.4f})\n")


4) Tradución

In [7]:
translator=pipeline('translation', model='Helsinki-NLP/opus-mt-es-en')
translated_reviews = [translator(comment, max_length=50)[0]['translation_text'] for comment in comentarios]

for original, translated in zip(comentarios, translated_reviews):
    print(f"Original: {original}")
    print(f"Translated: {translated}\n")

Your input_length: 178 is bigger than 0.9 * max_length: 50. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 305 is bigger than 0.9 * max_length: 50. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 132 is bigger than 0.9 * max_length: 50. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Original: trobo a faltar serveis d&#039;atenciÃ³/ajuda psicolÃ²gica
Translated: serviceis missing d=039;psychological care

Original: El sistema de gestiÃ³ de les prÃ ctiques Ã©s el que queda mÃ©s fluix amb diferÃ¨ncia
Translated: The management system of the most fluix amb differentiÃ ncia

Original: Falta un telÃ¨fon d&#039;atenciÃ³ a l&#039;estudiant, a vegades hi ha dubtes quÃ¨ es fa difÃ­cil explicar per correu. A mÃ©s, al telÃ¨fon la resposta Ã©s al moment per correu no. Trobo quÃ¨ un telÃ¨fon d&#039;atenciÃ³ al client seria de molta utilitat. \ Falten classes onlines, com per exemple, la UNIR ofereix classes magistrals online, que crec necessÃ ries per a l&#039;aprenantage. Ãs insuficient donar tan sols un PDF.
Translated: It is difficult to explain per correu. Moreover, to the telephone the answer is to the moment per correu no. Trobo what a telephone d=039; it is difficult to explain per correu.

Original: Estic satisfeta amb la formaciÃ³, a excepciÃ³ de la metodologia de tre

FIN

In [ ]:
nltk.data.path.append('C:\\Users\\asanchezbelb\\Desktop\\UOC\\XaviNPL\\nltk_data')
# Verificar que el recurso 'punkt' está disponible
try:
    nltk.data.find('tokenizers/punkt')
    print("Recurso 'punkt' está disponible.")
except LookupError:
    print("Recurso 'punkt' no está disponible.")

import spacy

# Cargar los modelos de idioma
nlp_es = spacy.load("es_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")
nlp_ca = spacy.load("ca_core_news_sm")


In [ ]:
# def tokenize_text(text):
#     doc = nlp_es(text)
#     tokens = [token.text for token in doc]
    
#     return tokens
#     doc = nlp(text)
#     tokens = [token.text for token in doc]
    
#     return tokens

# tokenized_data = [tokenize_text(doc) for doc in cleaned_data]

# # Imprimir los resultados
# for idx, tokens in enumerate(tokenized_data):
#     print(f"Documento {idx+1}: {tokens}")